In [3]:
import networkx as nx
import pandas as pd

In [16]:
G = nx.DiGraph()
for count, line in enumerate(open("data/wiki-topcats-reduced.txt")):
            grlst=line.replace("\n", "").split(sep="\t")
            G.add_edge(int(grlst[0]),int(grlst[1]))

In [4]:
#from txt file extract informations about graph edges
data = pd.read_csv('data\wiki-topcats-reduced.txt', sep="	", header=None)
data.columns = ["from", "to"]
list1 = data["from"].values
list2 = data["to"].values

#finding all the edges in our graph
set1=set(list1)
set2=set(list2)
fin_set = set1.union(set2)
print(len(fin_set))

461193


In [103]:
#from txt file extract informations about category
data_categories = []
with open('data/wiki-topcats-categories.txt') as f:
    for i, line in enumerate(f):
        tmp = line.split(';')
        
        #removing small categories
        if len(tmp[1]) > 3500:
            a=[tmp[0].replace('Category:', '')]
            a.append([int(i) for i in tmp[1].split()])
            data_categories.append(a)

f.close()


In [104]:
#cleaning categories from nodes out of our graph
cate = []
for i in range(len(data_categories)):
    data_categories[i][1] = fin_set.intersection(data_categories[i][1])
    if len(data_categories[i][1]) > 3500:
        cate.append(data_categories[i])
        
#cate: nested list, for each raw: 0 category name, 1 set of nodes in the category        
print(len(cate))

29


In [105]:
for i in range(len(cate)):
    print(len(cate[i][1]))

7538
7814
5097
3737
5827
4588
348300
5549
5192
6491
5568
4122
28498
7729
11660
4895
4760
4422
22463
15159
4614
11531
13865
7561
10759
4346
5532
3720
4025


In [106]:
rank = [4,18,25,12,3,24,19,27,7,13,20,1,0,11,21,26,14,9,23,2,28,10,8,15,22,5,17,16,6]

In [107]:
# clean ranked categories
for i in range(len(rank)):
    if i == 0:
        used_nodes = cate[rank[0]][1]
    else:
        cate[rank[i]][1] = cate[rank[i]][1].difference(used_nodes)
        used_nodes = used_nodes.union(cate[rank[i]][1])

In [108]:
cate[4][1]

{1671168,
 884755,
 884774,
 622637,
 1671214,
 1671219,
 622647,
 622649,
 557114,
 884802,
 622674,
 884826,
 622687,
 884831,
 622690,
 622706,
 622712,
 622714,
 884860,
 884862,
 884871,
 622729,
 884885,
 622742,
 884901,
 622765,
 884924,
 884930,
 884931,
 884932,
 884934,
 622801,
 884946,
 622804,
 884949,
 884953,
 1638626,
 622826,
 884973,
 884974,
 1638645,
 884986,
 1638651,
 884991,
 884993,
 885003,
 622867,
 622869,
 885026,
 622885,
 622886,
 1638712,
 885056,
 622915,
 885060,
 885070,
 1638737,
 622931,
 622933,
 885087,
 622953,
 622969,
 622973,
 1278339,
 622991,
 622992,
 885139,
 1278367,
 1638817,
 1638820,
 1278380,
 623032,
 1475041,
 1475047,
 164330,
 623085,
 623099,
 1638918,
 1638920,
 623114,
 1638929,
 623123,
 1671710,
 623152,
 1638961,
 1638963,
 623156,
 1638964,
 1638967,
 623179,
 623184,
 1638992,
 623214,
 623216,
 1639034,
 1639046,
 1639054,
 1639055,
 1213072,
 1639063,
 1639080,
 1639106,
 1639169,
 1639170,
 623364,
 623379,
 623381,
 16

In [109]:
for i in range(len(cate)):
    print(len(cate[i][1]))

2111
5554
3232
3706
5827
2875
277504
5011
5100
6423
5404
2128
28493
7729
11660
360
3850
637
22463
1763
4132
10011
4329
7561
6685
4338
2819
3692
4025


In [110]:
def weight_nodes (category):
    '''Takes in input a category as a set, returns a list with every node and its weight, 
    sorted according to the highest weights of the nodes (articles)'''
    #creating a vector of tuples which will contain the weight of nodes
    W = []
    for i in category:
        pred = set(G.predecessors(i))
        
        V = [i, len(pred)]        
        W.append(V)
    
    W = sorted(W,key=lambda x: x[1], reverse = True)  
    return W       

In [111]:
def weight_nodes2 (category, prev_wei):
    '''Takes in input a category as a set and the other as a list with the weights of internal nodes,
    returns a list with the internal weight plus the external given from the previous category, 
    sorted according to the highest weights of the nodes (articles)'''
    #calculate the internal weight
    W = weight_nodes(category)
    
    #add the weight given by the previous category
    for i in range(len(W)):
        a = list(G.predecessors(W[i][0]))
        
        #for each predecessor check if the node is in the previous category, 
            #if it is add the weight of the predecessor to the successor
        for y in range(len(a)):
            for j in range(len(prev_wei)):
                if a[y] == prev_wei[j][0]:
                    W[i][1] += prev_wei[j][1]
                    break
            
    #calculating the weight    
        
    W = sorted(W,key=lambda x: x[1], reverse = True)    
    return W 

In [114]:
# once ranked we assign each node to the highest ranked category they belong

#first category
weight = []

C = [cate[rank[0]][1]]

weight.append(weight_nodes(C[0]))


a = weight_nodes(C[0])
weight[0]

[[82393, 282],
 [89978, 257],
 [81878, 169],
 [81856, 163],
 [81568, 131],
 [78971, 124],
 [84849, 116],
 [81996, 110],
 [81829, 108],
 [83088, 108],
 [82326, 93],
 [81835, 92],
 [81858, 88],
 [82354, 86],
 [81905, 83],
 [88793, 80],
 [81940, 79],
 [83513, 76],
 [87589, 73],
 [82587, 70],
 [83955, 67],
 [81291, 62],
 [82350, 61],
 [83531, 60],
 [82368, 59],
 [1358669, 54],
 [81932, 53],
 [81219, 52],
 [82948, 52],
 [83081, 51],
 [89700, 51],
 [84847, 50],
 [83539, 48],
 [1358640, 46],
 [81587, 46],
 [81774, 45],
 [82624, 45],
 [86475, 45],
 [82337, 44],
 [84178, 44],
 [81493, 43],
 [82889, 42],
 [895726, 41],
 [81314, 41],
 [88982, 41],
 [191281, 41],
 [76811, 40],
 [82052, 40],
 [82343, 40],
 [82372, 38],
 [83542, 38],
 [89973, 38],
 [81213, 37],
 [81551, 37],
 [81826, 37],
 [81921, 37],
 [83516, 36],
 [89876, 36],
 [81925, 35],
 [83530, 35],
 [86509, 35],
 [81946, 34],
 [82321, 34],
 [89877, 34],
 [80813, 32],
 [81334, 32],
 [81565, 32],
 [88941, 32],
 [81163, 31],
 [734911, 30],
 [8

In [115]:
#others categories
for i in range(len(rank)):
    
    if i == 0:
        continue
    else:
        
        C = cate[rank[i]][1]
        w = weight_nodes2(C, weight[i-1])
        
        weight.append(w)
        

In [63]:
weight[2]

[[1488646, 235],
 [1247620, 227],
 [1065905, 192],
 [1772800, 192],
 [1774700, 184],
 [1247471, 181],
 [940252, 177],
 [1049581, 159],
 [1338693, 157],
 [1338691, 130],
 [1770904, 126],
 [1247452, 114],
 [1247105, 111],
 [1775003, 103],
 [1338692, 101],
 [1262203, 101],
 [1377045, 97],
 [1144563, 94],
 [1507882, 92],
 [1772763, 86],
 [1772811, 83],
 [1084345, 82],
 [658463, 82],
 [1024783, 81],
 [1627140, 81],
 [1246203, 78],
 [1322768, 76],
 [1263473, 74],
 [1074287, 70],
 [136327, 68],
 [1049619, 68],
 [1775000, 64],
 [589507, 62],
 [1632798, 62],
 [1322767, 62],
 [1338690, 60],
 [1246340, 60],
 [1772772, 59],
 [1322765, 57],
 [1268572, 56],
 [1356120, 54],
 [1772818, 53],
 [1247269, 52],
 [166051, 51],
 [1484763, 50],
 [1278950, 50],
 [551711, 50],
 [874803, 48],
 [1774502, 48],
 [1772764, 48],
 [1338948, 48],
 [1069508, 47],
 [1414824, 47],
 [1073565, 46],
 [149357, 46],
 [1049373, 45],
 [1772762, 45],
 [1037320, 45],
 [940549, 42],
 [1263885, 42],
 [437874, 42],
 [1338776, 41],
 [

In [65]:
weight[18]

[[953284, 113],
 [1227839, 50],
 [1181074, 34],
 [249059, 33],
 [1186668, 32],
 [1182555, 31],
 [1185218, 30],
 [1177531, 28],
 [1179366, 27],
 [1182556, 25],
 [146527, 25],
 [1184081, 23],
 [1184501, 22],
 [1175504, 21],
 [1226587, 20],
 [1183234, 19],
 [1185485, 19],
 [1184878, 17],
 [1176106, 17],
 [1185213, 16],
 [1183712, 15],
 [1183784, 15],
 [1184087, 15],
 [1176528, 15],
 [1180960, 14],
 [1184313, 14],
 [1184854, 14],
 [142933, 14],
 [1175366, 14],
 [658474, 13],
 [1183236, 13],
 [1184266, 13],
 [1184398, 13],
 [1184444, 13],
 [1184618, 13],
 [1185518, 13],
 [1186008, 13],
 [1186271, 13],
 [677205, 13],
 [1595276, 13],
 [710676, 13],
 [1178462, 13],
 [1178830, 13],
 [1180735, 12],
 [689385, 12],
 [689640, 12],
 [1182405, 12],
 [1184024, 12],
 [1184483, 12],
 [1184787, 12],
 [1185017, 12],
 [1229399, 12],
 [1198951, 12],
 [773001, 12],
 [1469818, 12],
 [1175573, 12],
 [1180814, 11],
 [1181641, 11],
 [1184504, 11],
 [1184572, 11],
 [400015, 11],
 [1198956, 11],
 [1198964, 11],
 [

In [66]:
weight[28]

[[1185127, 6234],
 [1400548, 3599],
 [1180117, 3200],
 [1400547, 2972],
 [1400635, 2597],
 [1061284, 2125],
 [1181772, 2009],
 [1184029, 1492],
 [1178721, 1478],
 [1179311, 1366],
 [1179931, 1272],
 [1184044, 1237],
 [1179873, 1195],
 [1176029, 1129],
 [1179886, 1111],
 [1180263, 1084],
 [1184368, 1084],
 [1184448, 1067],
 [1400483, 1040],
 [1184073, 1001],
 [1184022, 985],
 [1181769, 963],
 [1061960, 901],
 [539805, 886],
 [1400479, 869],
 [676816, 852],
 [1174956, 838],
 [1183901, 836],
 [1179622, 834],
 [1185207, 825],
 [1179263, 823],
 [1179838, 812],
 [1179291, 798],
 [1184226, 793],
 [1184210, 783],
 [1181013, 781],
 [1184225, 773],
 [1179147, 764],
 [1108354, 747],
 [1184864, 745],
 [1226552, 740],
 [1184026, 736],
 [539587, 728],
 [1184223, 722],
 [1179039, 715],
 [1184345, 703],
 [1061920, 701],
 [1184460, 690],
 [1184213, 666],
 [1062053, 656],
 [1180007, 653],
 [1400624, 648],
 [1179398, 642],
 [1179052, 642],
 [1183728, 641],
 [1179465, 639],
 [1176160, 636],
 [1176785, 616